# Week 10 Assignment: Groth16 Part I

src: <https://almondine-song-c43.notion.site/Homework-10-3195364280584ba09b22b459c9282bbf>

Use a QAP from an earlier homework as a starting point.

The trusted setup should generate the following values:

$$
\tau, \alpha, \beta
$$

which are the “toxic waste” (read this [https://www.coindesk.com/layer2/2022/01/26/what-is-zcash-the-privacy-coin-explained/](https://www.coindesk.com/layer2/2022/01/26/what-is-zcash-the-privacy-coin-explained/#:~:text=A%20trusted%20setup%20creates%20a,without%20revealing%20what%20it%20was)).

## Notation

Bold letters are vectors

$$
\langle\mathbf{a},\mathbf{b}\rangle
$$

Is an inner product between vectors **a** and **b**.

$$
[A]_1
$$

Is a G1 point

$$
[B]_2
$$

Is a G2 point

$$
I_{12}
$$

Is the identity in G12

Carry out the following computations in Python:

## Trusted Setup

Generate the powers of tau for A and B:

$$
[\tau^dG_1, \tau^{d-1}G_1,...,\tau G_1, G_1]
$$

$$
[\tau^dG_2, \tau^{d-1}G_2,...,\tau G_2, G_2]
$$

$$
[\dots,\tau^2t(\tau)G_1,\tau t(\tau)G_1,t(\tau)G_1]
$$

$$
\Psi_i=(w_i(\tau)+\alpha v_i(\tau)+\beta u_i(\tau))G_1 \space\space\space\text{for i = 1 to m}
$$

The trusted setup publishes

$$
\begin{align*}[\alpha]_1&=\alpha G_1 \\ [\beta]_2&=\beta G2 \\  \mathbf{\Psi}&=(w_i(\tau)+\alpha v_i(\tau)+\beta u_i(\tau))G_1\vert_{i=1}^m\end{align*}
$$

## Prover

The prover computes their witness vector **a** and computes:

$$
[A]_1 = [\alpha]_1 +\sum_{i=1}^m a_iu_i(\tau)
$$

$$
[B]_2 = [\beta]_2+\sum_{i=1}^m a_i v_i(\tau)
$$

$$
[C]_1 = \sum_{i=0}^m a_i\Psi_i+\langle \mathbf{h}, \mathbf{\eta} \rangle
$$

## Verifier

The verifier computes:

$$
\text{I}_{12}\stackrel{?}{=} \text{neg}([A]_1)\cdot [B]_2+[\alpha]_1\cdot [\beta]_2+[C]_1G_1
$$

If you get very stuck, feel free to refer to past implementations by students (which have been published on GitHub).

In [ ]:
import numpy as np
from functools import reduce
from py_ecc.bn128 import G1, G2, Z1, Z2, Z1, multiply, add, curve_order, pairing
import secrets
import galois

def trusted_setup(degree, GF, tau = None):
    if tau is None:
        # A rand between [1, ... GF_order - 1] inclusively
        tau = secrets.randbelow(GF.order - 1) + 1

    g1_pts = [multiply(G1, int((GF(tau) ** p))) for p in range(degree + 1)]
    g2_pts = [multiply(G2, int((GF(tau) ** p))) for p in range(degree + 1)]
    return (g1_pts, g2_pts)

def ecc_eval(poly: galois.Poly, g_pp, zero_g):
    coeffs = poly.coeffs
    degrees = poly.degrees
    result = zero_g
    for idx, deg in enumerate(degrees):
        result = add(result, multiply(g_pp[deg], int(coeffs[idx])))
    return result

def to_galois(n: int, GF: galois.FieldArray):
    if n >= 0 and n < GF.order:
        return GF(n)
    elif n < 0:
        positive = int(n)
        while positive < 0: positive += GF.order
        return GF(positive)

    # n > GF.order
    positive = int(n)
    while positive >= GF.order: positive -= GF.order
    return GF(positive)

def r1cs_to_qap(L: np.array, R: np.array, O: np.array, w: np.array, GF: galois.FieldArray) -> (np.array, np.array, np.array, np.array, np.array):
    """
    Convert an R1CS to QAP. All the computation is done in finite field
    They need to satisfy this relationship: L·w ⊙ R·w = O·w

    Paramters:
    L (np.array): m x n array, wrapped in Galois field
    R (np.array): m x n array, wrapped in Galois field
    O (np.array): m x n array, wrapped in Galois field
    w (np.array): n x 1 vector, wrapped in Galois field
    GF (galois.FieldArray): The finite field order

    Returns:

    Raises:
    """
    # Check for the params size
    if (L.ndim != 2 or R.ndim != 2 or O.ndim != 2 or w.ndim != 1
        or L.shape[0] != R.shape[0] or R.shape[0] != O.shape[0]
        or L.shape[1] != R.shape[1] or R.shape[1] != O.shape[1]
        or w.shape[0] != O.shape[1]
       ):
        raise Exception("Parameters sizes mismatched.")

    def interpolate_col(col):
        xs = GF(np.array(range(1, len(col) + 1)))
        return galois.lagrange_poly(xs, col)

    def inner_product_polys_with_w(polys, witness):
        mul_ = lambda x, y: x * y
        sum_ = lambda x, y: x + y
        return reduce(sum_, map(mul_, polys, witness))

    def get_t_poly(poly: galois.Poly) -> galois.Poly:
        """
        The return value is a polynomial with deg(poly) - 1
        """
        return galois.Poly.Roots(list(range(1, poly.degree)), field = GF)


    Lg = GF(L)
    Rg = GF(R)
    Og = GF(O)

    U = np.apply_along_axis(interpolate_col, 0, Lg)
    V = np.apply_along_axis(interpolate_col, 0, Rg)
    W = np.apply_along_axis(interpolate_col, 0, Og)

    # Uw·Vw = Ww + h(x)t(x)
    Uw = inner_product_polys_with_w(U, w)
    Vw = inner_product_polys_with_w(V, w)
    Ww = inner_product_polys_with_w(W, w)

    # Uw·Vw - Ww
    result_poly = Uw * Vw - Ww
    T = get_t_poly(result_poly)
    H = result_poly // T

    return (Uw, Vw, Ww, H, T)

def main():
    # Define the Galois field
    print(f"Initializing galois field...")

    # For testing, switch to use a smaller field as below:
    # GF = galois.GF(79)
    GF = galois.GF(curve_order)

    # Formula
    # out = 3x²y + 5xy - x - 2y + 3
    # witness = {x: 100, y: 100}
    
    # Define the matrices
    L = np.array([[0,0,3,0,0,0],
                  [0,0,0,0,1,0],
                  [0,0,1,0,0,0]], dtype="object")

    R = np.array([[0,0,1,0,0,0],
                  [0,0,0,1,0,0],
                  [0,0,0,5,0,0]], dtype="object")

    O = np.array([[0,0,0,0,1,0],
                  [0,0,0,0,0,1],
                  [-3,1,1,2,0,-1]], dtype="object")

    print(f"Computing witness and Lg, Rg, Og...")

    # Define the witness
    x = to_galois(100, GF)
    y = to_galois(100, GF)
    v1 = to_galois(3, GF) * x * x
    v2 = v1 * y
    out = v2 + to_galois(5, GF) * x * y - x - to_galois(2, GF) * y + to_galois(3, GF)
    witness = GF(np.array([1, out, x, y, v1, v2]))

    # Convert ndarray into Galois field
    Lg = GF(np.array([[to_galois(x, GF) for x in row] for row in L], dtype="object"))
    Rg = GF(np.array([[to_galois(x, GF) for x in row] for row in R], dtype="object"))
    Og = GF(np.array([[to_galois(x, GF) for x in row] for row in O], dtype="object"))

    print(f"Converting r1cs to qap...")

    U, V, W, H, T = r1cs_to_qap(Lg, Rg, Og, witness, GF)
    HT = H * T

    print(f"Performing trusted setup...")

    # HT polynomial has the highest degree no. among (U, V, W, HT)
    g1_pp, g2_pp = trusted_setup(HT.degree, GF)

    print(f"Performing ecc computation on U, V, W, HT")

    Ug1 = ecc_eval(U, g1_pp, Z1)
    Vg2 = ecc_eval(V, g2_pp, Z2)
    Wg1 = ecc_eval(W, g1_pp, Z1)
    HTg1 = ecc_eval(HT, g1_pp, Z1)

    print(f"Performing 2 ecc pairings...")

    # Check if pairing (Vg2, Ug1) == pairing(G2, Wg1 + HTg1)
    lhs = pairing(Vg2, Ug1)
    rhs = pairing(G2, add(Wg1, HTg1))
    assert lhs == rhs, "pairing (Ug1, Vg2) != pairing(Wg1 + HTg1, G2)"

main()